# 第12章 动态内存

- C++支持动态分配对象，动态分配的对象的生存期与它们在哪里创建是无关的，只有当显式地被释放时，这些对象才会销毁
- 动态对象的正确释放被证明是编程中极其容易出错的地方，为了更安全地使用动态对象，标准库定义了两个智能指针类型来管理动态分配的对象，当一个对象应该被释放时，指向它的智能指针可以确保自动地释放它

- 我们的程序到目前为止只使用过静态内存或栈内存；静态内存用来保存局部static对象、类static数据成员以及定义在任何函数之外的变量；栈内存用来保存定义在函数内的非static对象；分配在静态或栈内存中的对象由编译器自动创建和销毁；对于栈对象，仅在其定义的程序块运行时才存在，static对象在使用之前分配，在程序结束时销毁
- 除了静态内存和栈内存，每个程序还拥有一个内存池，这部分内存被称作自由空间（free store）或堆（heap）；程序用堆来存储动态分配（dynamically allocate）的对象——即，那些在程序运行时分配的对象；动态对象的生存期由程序来控制，也就是说，当动态对象不再使用时，我们的代码必须显式地销毁他们

## 12.1 动态内存与智能指针

- 在C++中，动态内存的管理是通过一对运算符来完成的：`new`，在动态内存中为对象分配空间并返回一个指向该对象的指针，我们可以选择对对象进行初始化；`delete`，接受一个动态对象的指针，销毁该对象，并释放与之关联的内存
- 动态内存的使用很容易出问题，因为确保在正确的时间释放内存是极其困难的；有时我们会忘记释放内存，在这种情况下就会产生内存泄漏；有时在尚有指针引用内存的情况下我们就释放了它，在这种情况下就会产生引用非法内存的指针
- 为了更容易（同时也更安全）地使用动态内存，新的标准库提供了两种智能指针（smart pointer）类型来管理动态对象；智能指针的行为类似常规指针，重要的区别是它负责自动释放所指向的对象；新标准库提供的这两种智能指针的区别在于管理底层指针的方式：`shared_ptr`允许多个指针指向同一个对象，`unique_str`则“独占”所指向的对象，标准库还定义了一个名为`weak_ptr`的伴随类，它是一种弱引用，指向`shared_ptr`所管理的对象；这三种类型都定义在memory头文件中

### 12.1.1 shared_ptr类

- 类似vector，智能指针也是模板，因此，当我们创建一个智能指针时，必须提供额外的信息——指针可以指向的类型，与vector一样，我们在尖括号内给出类型，之后是所定义的这种智能指针的名字：
```c++
shared_ptr<string> p1; // shared_ptr，可以指向string
shared_ptr<list<int>> p2; // shared_ptr，可以指向int的list
```
- 默认初始化的智能指针中保存着一个空指针；智能指针的使用方式与普通指针类似，解引用一个智能指针返回它指向的对象，如果在一个条件判断中使用智能指针，效果就是检测它是否为空

#### make_shared函数

- 最安全的分配和使用动态内存的方法是调用一个名为make_shared的标准库函数，此函数在动态内存中分配一个对象并初始化它，返回指向此对象的shared_ptr
```c++
// 指向一个值为42的int的shared_ptr
shared_ptr<int> p3 = make_shared<int>(42);
// p4指向一个值为“999999999”的string
shared_ptr<string> p4 = make_shared<string>(10, '9');
// p5指向一个值初始化的int，即，值为0
shared_ptr<int> p5 = make_shared<int>();
```
- 当然，我们通常用auto定义一个对象来保存make_shared的结果，这种方式比较简单：
```c++
// p6指向一个动态分配的空vector<string>
auto p6 = make_shared<vector<string>>();
```

#### shared_ptr的拷贝和赋值

- 当进行拷贝或赋值操作时，每个shared_ptr都会记录有多少个其他shared_ptr指向相同的对象：
```c++
auto p = make_shared<int>(42); // p指向的对象只有p一个引用者
auto q(p); // p和q指向相同对象，此对象有两个引用者
```
- 我们可以认为每个shared_ptr都有一个关联的计数器，通常称其为引用计数（reference count），无论何时我们拷贝一个shared_ptr，计数器都会递增；例如，当用一个shared_ptr初始化另一个shared_ptr，或将它作为参数传递给一个函数以及作为函数的返回值时，它所关联的计数器就会递增；当我们给shared_ptr赋予一个新值或是shared_ptr被销毁（例如一个局部的shared_ptr离开其作用域）时，计数器就会递减
- 一旦一个shared_ptr的计数器变为0，它就会自动释放自己所管理的对象
```c++
auto r = make_shared<int>(42); // r指向的int只有一个引用者
r = q; // 给r赋值，令它指向另一个地址；递增q指向的对象的引用计数；递减r原来指向的对象的引用计数；r原来指向的对象已没有引用者，会自动释放
```

#### shared_ptr自动销毁所管理的对象

- 当指向一个对象的最后一个shared_ptr被销毁时，shared_ptr类会自动销毁此对象；它是通过另一个特殊的成员函数——析构函数（destructor）完成销毁工作的；类似于构造函数，每个类都有一个析构函数，就像构造函数控制初始化一样，析构函数控制此类型的对象销毁时做什么操作
- shared_ptr的析构函数会递减它所指向的对象的引用计数，如果引用计数变为0，shared_ptr的析构函数就会销毁对象，并释放它占用的内存
- 由于在最后一个shared_ptr销毁前内存都不会释放，保证shared_ptr在无用之后不再保留就非常重要了，如果你忘记了销毁程序不再需要的shared_ptr，程序仍会正确执行，但会浪费内存

#### 使用了动态生存期的资源的类

- 程序使用动态内存出于以下三种原因之一：1.程序不知道自己需要使用多少对象；2.程序不知道所需对象的准确类型；3.程序需要在多个对象间共享数据

### 12.1.2 直接管理内存

- C++语言定义了两个运算符来分配和释放内存，运算符new分配内存，delete释放new分配的内存；相对于智能指针，使用这两个运算符管理内存非常容易出错

#### 使用new动态分配和初始化对象

- 在自由空间分配的内存是无名的，因此new无法为其分配的对象命名，而是返回一个指向该对象的指针：
```c++
int *pi = new int; // pi指向一个动态分配的、未初始化的无名对象
```
- 默认情况下，动态分配的对象是默认初始化的，这意味着内置类型或组合类型的对象的值将是未定义的，而类类型对象将用默认构造函数进行初始化：
```c++
string *ps = new string; // 初始化为空string
int *pi = new int; // pi指向一个未初始化的int
```
- 我们可以使用直接初始化方式来初始化一个动态分配的对象；我们可以使用传统的构造方式（使用圆括号），在新标准下，也可以使用列表初始化（使用花括号）：
```c++
int *pi = new int(1024); // pi指向的对象的值为1024
string *ps = new string(10, '9'); // *ps为“999999999”
vector<int> = new vector<int>{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}; // vector有10个元素，值依次从0到9
```
- 也可以对动态分配的对象进行值初始化，只需在类型名之后跟一对空括号即可：
```c++
string *ps1 = new string; // 默认初始化为空string
string *ps = new string(); // 值初始化为空string
int *pi1 = new int; // 默认初始化；*pi1的值未定义
int *pi2 = new int(); // 值初始化为0；*pi2为0
```
- 对于定义了自己的构造函数的类类型（例如string）来说，要求值初始化是没有意义的，不管采用什么形式，对象都会通过默认构造函数来初始化；但对于内置类型，两种形式的差别就很大了，值初始化的内置类型对象有着良好定义的值，而默认初始化的对象的值则是未定义的

- 如果我们提供了一个括号包围的初始化器，就可以使用auto从此初始化器来推断我们想要分配的对象的类型，但是，由于编译器要用初始化器的类型来推断要分配的类型，只有当括号中仅有单一初始化器时才可以使用auto：
```c++
auto p1 = new auto(obj); // p指向一个与obj类型相同的对象，该对象用obj进行初始化
auto p2 = new auto{a,b,c}; // 错误：括号中只能有单个初始化器
```

#### 内存耗尽

```c++
int *p1 = new int; // 如果分配失败，new抛出std::bad_alloc
int *p2 = new (nothrow) int; // 如果分配失败，new返回一个空指针
```

#### 释放动态内存

```c++
delete p; // p必须指向一个动态分配的对象或是一个空指针
```
- 与new类型类似，delete表达式也执行两个动作：销毁给定的指针指向的对象，释放对应的内存

#### 指针值和delete

- 我们传递给delete的指针必须指向动态分配的内存，或者是一个空指针；释放一块并非new分配的内存，或者将相同的指针值释放多次，其行为是未定义的

#### 动态对象的生存期直到被释放时为止

- 由shared_ptr管理的内存在最后一个shared_ptr销毁时会被自动释放，但对于通过内置指针类型来管理的内存，就不是这样了，对于一个由内置指针管理的动态对象，直到被显式释放之前它都是存在的
- 返回指向动态内存的指针（而不是智能指针）的函数给其调用者增加了一个额外负担——调用者必须记得释放内存
- 使用new和delete管理动态内存存在三个常见问题：1.忘记delete内存（内存泄漏）；2.使用已经释放掉的对象（在释放内存后将指针置为空，有时可检测出这种错误）；3.同一块内存释放两次（当有两个指针指向相同的动态分配对象时，可能发生）
- 当我们delete一个指针后，指针值就变为无效了，虽然指针已经无效，但在很多机器上指针仍然保存着（已经释放了的）动态内存的地址；在delete之后，指针就变成了空悬指针（dangling pointer），即，指向一块曾经保存数据对象但现在已经无效的内存的指针；一种解决办法是在delete之后将nullptr赋予指针

### 12.1.3 shared_ptr和new结合使用

- 如前所述，如果我们不初始化一个智能指针，它就会被初始化为一个空指针，我们还可以用new返回的指针来初始化智能指针：
```c++
shared_ptr<double> p1; // shared_ptr可以指向一个double
shared_ptr<int> p2(new int(42)); // p2指向一个值为42的int
```
- 默认情况下，一个用来初始化智能指针的普通指针必须指向动态内存，因为智能指针默认使用delete释放它所关联的对象，我们可以将智能指针绑定到一个指向其它类型的资源的指针上，但是为了这样做，必须提供自己的操作来替代delete

- 当将一个shared_ptr绑定到一个普通指针时，我们就将内存的管理责任交给了这个shared_ptr，一旦这样做了，我们就不应该再使用内置指针来访问shared_ptr所指向的内存了；使用一个内置指针来访问一个智能指针所负责的对象是很危险的，因为我们无法知道这个对象何时被销毁

- 智能指针类型定义了一个名为get的函数，它返回一个内置指针，指向智能指针管理的对象；此函数是为了这样一种情况而设计的：我们需要向不能使用智能指针的代码传递一个内置指针；使用get返回的指针的代码不能delete此指针

### 12.1.4 智能指针和异常

```c++
void f() {
    shared_ptr<int> sp(new int(42));
    // 这段代码抛出一个异常，且在f中未捕获
} // 在函数结束时shared_ptr自动释放内存
```
```c++
void f() {
    int *ip =  new int(42);
    // 这段代码抛出一个异常，且在f中未捕获
    delete ip; // 手动释放内存
} // 若new和delete之间发生异常，且未捕获，则内存不会被释放
```

#### 智能指针陷阱

- 智能指针可以提供对动态分配的内存安全而又方便的管理，但这建立在正确使用的前提下，为了正确使用智能指针，我们必须坚持一些基本规范：1.不适用相同的内置指针值初始化（或reset）多个智能指针；2.不delete get()返回的指针；3.不使用get()初始化或reset另一个智能指针；4.如果你使用get()返回的指针，记住当最后一个对应的智能指针销毁后，你的指针就变为无效了；5.如果你使用智能指针管理的资源不是new分配的内存，记住传递给它一个删除器

### 12.1.5 unique_ptr

- 一个unique_ptr“拥有”它所指向的对象，与shared_ptr不同，某个时刻只能有一个unique_ptr指向一个给定对象，当unique_ptr被销毁时，它所指向的对象也被销毁
- 与shared_ptr不同，没有类似make_shared的标准库函数返回一个unique_ptr，当我们定义一个unique_ptr时，需要将其绑定到一个new返回的指针上，类似shared_ptr，初始化unique_ptr必须采用直接初始化形式
```c++
unique_ptr<double> p1; // 可以指向一个double的unique_ptr
unique_ptr<int> p2(new int(42)); // p2指向一个值为42的int
```
- 由于一个unique_ptr拥有它指向的对象，因此unique_ptr不支持普通的拷贝或赋值操作

- 虽然我们不能拷贝或赋值unique_ptr，但可以通过调用release或reset将指针的所有权从一个（非const）unique_ptr转移给另一个unique
- 通常release会切断unique_ptr和它原来管理的对象间的联系，release返回的指针通常被用来初始化另一个智能指针或给另一个智能指针赋值，如果我们不用另一个智能指针来保存release返回的指针，我们的程序就要负责资源的释放：
```c++
p2.release(); // 错误，p2不会释放内存，而且我们丢失了指针
auto p = p2.release(); // 正确，但我们必须记得delete(p)
```

- 不能拷贝unique_ptr的规则有一个例外：我们可以拷贝或赋值一个将要被销毁的unique_ptr
```c++
// 从函数直接返回一个unique_ptr
unique_ptr<int> clone(int p) {
    return unique_ptr<int> (new int(p));
}
// 返回一个局部对象的拷贝
unique_ptr<int> clone(int p) {
    unique_ptr<int> ret(new int(p));
    return ret;
}
```
- 编译器知道要返回的对象将要被销毁，在此情况下，编译器执行一种特殊的拷贝

### 12.1.6 weak_ptr

- weak_ptr是一种不控制所指向对象生存期的智能指针，它指向由一个shared_ptr管理的对象；将一个weak_ptr绑定到一个shared_ptr不会改变shared_ptr的引用计数，一旦最后一个指向对象的shared_ptr被销毁，对象就会被释放，即使有weak_ptr指向对象，对象也还是会被释放

## 12.2 动态数组

### 12.2.1 new和数组

- 为了让new分配一个对象数组，我们要在类型名之后跟一对方括号，在其中指明要分配的对象数目，在下例中，new分配要求数量的对象并（假定分配成功后）返回指向第一个对象的指针：
```c++
int *pia = new int[get_size()]; // pia指向第一个int
```
- 当用new分配一个数组时，我们并未得到一个数组类型的对象，而是得到一个数组元素类型的指针

#### 初始化动态分配对象的数组

- 默认情况下，new分配的对象，不管是单个分配的还是数组中的，都是默认初始化的，可以对数组中的元素进行值初始化，方法是在大小之后跟一对空括号
```c++
int *pia = new int[10]; // 10个未初始化的int
int *pia2 = new int[10](); // 10个值初始化为0的int
string *psa = new string[10]; // 10个空string
string *psa2 = new string[10](); // 10个空string
```
- 在新标准中，我们还可以提供一个元素初始化器的花括号列表
```c++
// 10个int分别用列表中对应的初始化器初始化
int *pia3 = new int[10]{0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
// 10个string，前4个用给定的初始化器初始化，剩余的进行值初始化
string *psa3 = new string[10]{"a", "an", "the", string(3, 'x')};
```
- 虽然我们用空括号对数组中的元素进行值初始化，但不能在括号中给出初始化器，这意味着不能用auto分配数组

#### 动态分配一个空数组是合法的

- 虽然我们不能创建一个大小为0的静态数组对象，但是当我们用new分配一个大小为0的数组时，new返回一个合法的非空指针，此指针保证与new返回的其它任何指针都不相同，对于零长度的数组来说，此指针就像尾后指针一样，我们可以像使用尾后迭代器一样使用这个指针，但此指针不能解引用——毕竟它不指向任何元素

#### 释放动态数组

- 为了释放动态数组，我们使用一种特殊形式的delete——在指针前加上一个空方括号对：
```c++
delete p; // p必须指向一个动态分配的对象或为空
delete [] pa; // pa必须指向一个动态分配的数组或为空；销毁pa指向的数组中的元素，并释放对应的内存，数组中的元素按逆序销毁
```
- 当我们释放一个指向数组的指针时，空方括号对是必须的：它指示编译器此指针指向一个对象数组的第一个元素；如果我们在delete一个指向数组的指针时忽略了方括号（或者在delete一个指向单一对象的指针时使用了方括号），其行为是未定义的

#### 智能指针和动态数组

- 标准库提供了一个可以管理new分配的数组的unique_ptr版本，为了用一个unique_ptr管理动态数组，我们必须在对象类型后面跟一对空括号：
```c++
// up指向一个包含10个未初始化int的数组
unique_ptr<int []> up(new int[10]);
up.release(); // 自动用delete[]销毁其指针
```

### 12.2.2 allocator类

- new有一些灵活性上的局限，其中一方面表现在它将内存分配和对象构造组合在了一起，类似的，delete将对象析构和内存释放组合在了一起；我们分配单个通常希望将内存分配和对象初始化组合在一起，因为在这种情况下，我们几乎肯定知道对象应有什么值；当分配一大块内存时，我们通常计划在这块内存上按需构造对象，在此情况下，我们希望将内存分配和对象构造分离，这意味着我们可以分配大块内存，但只在真正需要时才真正执行对象创建操作（同时付出一定开销）
- 一般情况下，将内存分配和对象构造组合在一起可能会导致不必要的浪费，例如：new表达式分配并初始化了n个string，但是，我们可能不需要n个string，少量string可能就足够了，这样，我们就可能创建了一些永远也用不到的对象，而且，对于那些确实要使用的对象，我们也在初始化之后立即赋予了它们新值，每个使用到的元素都被赋值了两次：第一次是在默认初始化时，随后是在赋值时；更重要的是，那些没有默认构造函数的类就不能动态分配数组了

- 标准库allocator类定义在头文件memory中，它帮助我们将内存分配和对象构造分离开来，它提供一种类型感知的内存分配方法，它分配的内存是原始的、未构造的